# Clean the S&P financial dataset

In [2]:
# import packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import preprocessing
from scipy import stats
import statsmodels.api as sm

## Import and read financial data

In [3]:
# direct to financials excel
excel_file = "../input_data/financials.xls"

In [4]:
# select revenue sheet and form dataset
sheet_name = "TOTAL REVENUE"
start_row = 13  
columns_to_read = "A:M"

df_revenue = pd.read_excel(excel_file, sheet_name=sheet_name, header=start_row, usecols=columns_to_read)
df_revenue

,Company Name,CY2016 Total Revenue,CY2017 Total Revenue,CY2018 Total Revenue,CY2019 Total Revenue,CY2020 Total Revenue,CY2021 Total Revenue,CY2022 Total Revenue,FY2023 Revenue (Capital IQ),FY2024 Revenue (Capital IQ),FY2025 Revenue (Capital IQ),FY2026 Revenue (Capital IQ),FY2027 Revenue (Capital IQ)
0,ABB Ltd (SWX:ABBN),24929,25196,27662,27978,26134,28945,29446,32174.52,32970.35,34510.78,36835,38700.25
1,"Amazon.com, Inc. (NasdaqGS:AMZN)",135987,177866,232887,280522,386064,469822,513983,570309.44,635092.39,709648.49,791246.88,880276.69
2,"Cisco Systems, Inc. (NasdaqGS:CSCO)",48570,48096,50825,51550,48026,51549,53161,56805.15,57918.8,59292.33,60879.18,64254
3,Intel Corporation (NasdaqGS:INTC),59387,62761,70848,71965,77867,79024,63054,53759.07,60727.03,65743.63,74040,79779
4,Kyocera Corporation (TSE:6971),9395.8,10414.5,11040.1,10770.1,10076.3,11949.3,13476.4,13419.54,13560.87,14163.88,14720.26,15586.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,"Veoneer, Inc.",2218,2322,2228,1902,1373,1657,-,-,-,-,-,-
268,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",1591.2,2740.7,3096.5,3465.7,5167.6,6756.4,6371.4,8270.25,9718.26,11490.58,10273.06,11373.39
269,Giesecke+Devrient Currency Technology Gb Ltd,-,-,-,-,-,-,-,-,-,-,-,-
270,Lutron Electronics Gmbh,-,-,-,-,-,-,-,-,-,-,-,-


In [5]:
# select ebitda sheet and form dataset
sheet_name = "EBITDA"
start_row = 13  
end_row = 285 
columns_to_read = "A:M" 

df_ebitda = pd.read_excel(excel_file, sheet_name=sheet_name, header=start_row, usecols=columns_to_read)
df_ebitda

,Company Name,CY2016 EBITDA,CY2017 EBITDA,CY2018 EBITDA,CY2019 EBITDA,CY2020 EBITDA,CY2021 EBITDA,CY2022 EBITDA,FY2023 EBITDA (Capital IQ),FY2024 EBITDA (Capital IQ),FY2025 EBITDA (Capital IQ),FY2026 EBITDA (Capital IQ),FY2027 EBITDA (Capital IQ)
0,ABB Ltd (SWX:ABBN),2987,2929,3227,3347,2668,4641,4477,5807.68,5943.65,6271.24,6997,7210
1,"Amazon.com, Inc. (NasdaqGS:AMZN)",12302,15584,27762,36330,48079,59312,54169,105145.67,124000.76,144358.23,174558,-
2,"Cisco Systems, Inc. (NasdaqGS:CSCO)",14721,14460,15307,16205,15236,15990,15811,20483.8,21102.33,21682.51,21951.15,-
3,Intel Corporation (NasdaqGS:INTC),22795,26563,32329,33254,36115,33874,15371,11438.91,17852.44,22840.69,27543,31629
4,Kyocera Corporation (TSE:6971),1230.6,1534,805.7,1423.8,1049.6,1742.2,1789.4,1686.95,1758.2,2033.8,2271.67,2601.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,"Veoneer, Inc.",84,58,1373,-345,-294,-226,-,-,-,-,-,-
268,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",9.4,135.8,5167.6,289.1,487.1,599.1,388,854.16,1034.16,1271.71,1220.11,1434.92
269,Giesecke+Devrient Currency Technology Gb Ltd,-,-,-,-,-,-,-,-,-,-,-,-
270,Lutron Electronics Gmbh,-,-,-,-,-,-,-,-,-,-,-,-


In [6]:
# select cogs sheet and form dataset
sheet_name = "Cost of Goods Sold"
start_row = 13  
end_row = 285 
columns_to_read = "A:H" 

df_cogs = pd.read_excel(excel_file, sheet_name=sheet_name, header=start_row, usecols=columns_to_read)
df_cogs

,Company Name,CY2016 Cost Of Goods Sold,CY2017 Cost Of Goods Sold,CY2018 Cost Of Goods Sold,CY2019 Cost Of Goods Sold,CY2020 Cost Of Goods Sold,CY2021 Cost Of Goods Sold,CY2022 Cost Of Goods Sold
0,ABB Ltd (SWX:ABBN),17270,17278,19059,19018,18123,19407,19712
1,"Amazon.com, Inc. (NasdaqGS:AMZN)",88265,111934,139156,165536,233307,272344,288831
2,"Cisco Systems, Inc. (NasdaqGS:CSCO)",17663,18121,19225,18575,17206,18918,20242
3,Intel Corporation (NasdaqGS:INTC),22767,23663,27111,29825,34255,35209,36188
4,Kyocera Corporation (TSE:6971),6981.16,7558.96,8281.95,7741.86,7430.67,8595.52,9663.65
...,...,...,...,...,...,...,...,...
267,"Veoneer, Inc.",1794,1857,1798,1591,1191,1384,-
268,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",1266.78,2176.07,2350.48,2532.98,3971.79,5345.76,5039.37
269,Giesecke+Devrient Currency Technology Gb Ltd,-,-,-,-,-,-,-
270,Lutron Electronics Gmbh,-,-,-,-,-,-,-


## Identify and remove missing values

In [7]:
# identify rows in cogs sheet with missing values
mask_cogs = (df_cogs == "-").any(axis = 1)
sum(mask_cogs)

58

In [8]:
# retrieve rows with missing values
filtered_rows_cogs = df_cogs[mask_cogs]

# select the 'company' column from the filtered rows
company_names_cogs = filtered_rows_cogs['Company Name']
company_names_cogs

13                    Brocade Communications Systems LLC
17         Harman International Industries, Incorporated
23                           Lexmark International, Inc.
30                                          Tellabs Inc.
34                                   Finisar Corporation
37                                 L3 Technologies, Inc.
48                                     IDEMIA France SAS
55                Otis Worldwide Corporation (NYSE:OTIS)
56                           ARRIS International Limited
57                                      Bose Corporation
97                          Varian Medical Systems, Inc.
101                       Fiat Chrysler Automobiles N.V.
108                         Danfoss Power Solutions Inc.
128                                    Collins Aerospace
129               Carrier Global Corporation (NYSE:CARR)
132                              Lite-On Group Co., Ltd.
140              YASKAWA Electric Corporation (TSE:6506)
144                            

In [9]:
# drop from all datasets the rows with missing values from the cogs sheet
df_revenue = df_revenue[~mask_cogs]
df_ebitda = df_ebitda[~mask_cogs]
df_cogs = df_cogs[~mask_cogs]

In [10]:
# identify rows in ebitda sheet before 2023 with missing values
mask_ebitda = (df_ebitda.iloc[:, :8] == "-").any(axis = 1)
filtered_rows_ebitda = df_ebitda[mask_ebitda]
company_names_ebitda = filtered_rows_ebitda['Company Name']
company_names_ebitda

131             Robert Bosch GmbH
152    Diehl Verwaltungs-Stiftung
Name: Company Name, dtype: object

In [11]:
df_revenue = df_revenue[~mask_ebitda]
df_ebitda = df_ebitda[~mask_ebitda]
df_cogs = df_cogs[~mask_ebitda]

## Revenue - use OLS to predict missing values for 2023-2027

In [12]:
# set the column names for revenue
col_names = ['Company name', '2016', '2017', '2018', '2019', '2020', '2021', '2022', "2023", "2024", "2025", "2026", "2027"]

df_revenue.columns = col_names
df_revenue

,Company name,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
0,ABB Ltd (SWX:ABBN),24929,25196,27662,27978,26134,28945,29446,32174.52,32970.35,34510.78,36835,38700.25
1,"Amazon.com, Inc. (NasdaqGS:AMZN)",135987,177866,232887,280522,386064,469822,513983,570309.44,635092.39,709648.49,791246.88,880276.69
2,"Cisco Systems, Inc. (NasdaqGS:CSCO)",48570,48096,50825,51550,48026,51549,53161,56805.15,57918.8,59292.33,60879.18,64254
3,Intel Corporation (NasdaqGS:INTC),59387,62761,70848,71965,77867,79024,63054,53759.07,60727.03,65743.63,74040,79779
4,Kyocera Corporation (TSE:6971),9395.8,10414.5,11040.1,10770.1,10076.3,11949.3,13476.4,13419.54,13560.87,14163.88,14720.26,15586.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,Fortive Corporation (NYSE:FTV),5378.2,5756.1,3800.4,4563.9,4634.4,5254.7,5825.7,6054.56,1639.34,6742.66,7979,8790
261,Xiaomi Corporation (SEHK:1810),9357.5,15673.5,23917.4,28145.8,33619,44892.1,38292.4,37203.67,2419.43,46869.64,47776.38,50536.69
262,Adient plc (NYSE:ADNT),16583,16391,17393,16304,12582,13312,14340,15436.42,15940.78,16616.28,17489.33,18106.4
266,"Resideo Technologies, Inc. (NYSE:REZI)",4455,4519,4827,4988,5071,5846,6370,6264.23,6440.96,754.11,-,-


In [13]:
# reframe revenue dataframe to get company, year, revenue columns
ols_revenue_data = df_revenue[['Company name', '2016', '2017', '2018', '2019', '2020', '2021', '2022']]
new_df_revenue = []
for x in ols_revenue_data["Company name"]:
    for y in list(ols_revenue_data.columns)[1:]:
        row = [x, y, ols_revenue_data[["Company name", y]][(ols_revenue_data["Company name"] == x)].values[0][1]]
        new_df_revenue.append(row)

new_df_revenue = pd.DataFrame(new_df_revenue, columns = ["Company", "Year", "Revenue"])
new_df_revenue

,Company,Year,Revenue
0,ABB Ltd (SWX:ABBN),2016,24929.0
1,ABB Ltd (SWX:ABBN),2017,25196.0
2,ABB Ltd (SWX:ABBN),2018,27662.0
3,ABB Ltd (SWX:ABBN),2019,27978.0
4,ABB Ltd (SWX:ABBN),2020,26134.0
...,...,...,...
1479,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2018,3096.5
1480,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2019,3465.7
1481,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2020,5167.6
1482,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2021,6756.4


In [14]:
# fit OLS to predict the missing values from 2023-2027 in the revenue sheet
new_df_revenue['Year'] = new_df_revenue['Year'].astype(int)  # Convert 'Year' to integers
new_df_revenue['Value'] = pd.to_numeric(new_df_revenue['Revenue'], errors='coerce')  # Convert 'Value' to numeric, handle non-numeric values as NaN
grouped_rev = new_df_revenue.groupby('Company')
ols_models = {}
for name, group in grouped_rev:
    X = sm.add_constant(group['Year'])  # Add a constant term for the intercept
    y = group['Revenue']

    # Fit the OLS model
    model = sm.OLS(y, X).fit()

    # Store the model in the dictionary with the company name as the key
    ols_models[name] = model

In [15]:
# predict revenue from 2023-2027 using the OLS model
companies = df_revenue["Company name"]
companies = companies.tolist()
new_rev = []
for company, group in grouped_rev:
    model = ols_models.get(company)
    year_prediction = []
    for year in [2023, 2024, 2025, 2026, 2027]:
        prediction = model.predict([1, year])  # Use .predict() to make predictions
        year_prediction.append(prediction[0])  # The prediction is a one-element array, so extract the value
    new_rev.append(year_prediction)

# show the predicted revenue dataframe
new_rev = pd.DataFrame(new_rev, columns = ["2023", "2024", "2025", "2026", "2027"])
new_rev.insert(0, "Company name", companies)
new_rev

,Company name,2023,2024,2025,2026,2027
0,ABB Ltd (SWX:ABBN),11582.442857,11747.596429,11912.750000,12077.903571,12243.057143
1,"Amazon.com, Inc. (NasdaqGS:AMZN)",29973.000000,30670.178571,31367.357143,32064.535714,32761.714286
2,"Cisco Systems, Inc. (NasdaqGS:CSCO)",6214.242857,6539.882143,6865.521429,7191.160714,7516.800000
3,Intel Corporation (NasdaqGS:INTC),23886.400000,26358.014286,28829.628571,31301.242857,33772.857143
4,Kyocera Corporation (TSE:6971),15934.900000,16560.810714,17186.721429,17812.632143,18438.542857
...,...,...,...,...,...,...
207,Fortive Corporation (NYSE:FTV),2715.014286,2723.189286,2731.364286,2739.539286,2747.714286
208,Xiaomi Corporation (SEHK:1810),15973.628571,16425.421429,16877.214286,17329.007143,17780.800000
209,Adient plc (NYSE:ADNT),6073.714286,6454.464286,6835.214286,7215.964286,7596.714286
210,"Resideo Technologies, Inc. (NYSE:REZI)",1614.042857,1730.771429,1847.500000,1964.228571,2080.957143


In [16]:
# merge the two datasets
merged_rev = pd.merge(ols_revenue_data, new_rev, on='Company name', how='inner')
merged_rev

,Company name,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
0,ABB Ltd (SWX:ABBN),24929,25196,27662,27978,26134,28945,29446,11582.442857,11747.596429,11912.750000,12077.903571,12243.057143
1,"Amazon.com, Inc. (NasdaqGS:AMZN)",135987,177866,232887,280522,386064,469822,513983,29973.000000,30670.178571,31367.357143,32064.535714,32761.714286
2,"Cisco Systems, Inc. (NasdaqGS:CSCO)",48570,48096,50825,51550,48026,51549,53161,6214.242857,6539.882143,6865.521429,7191.160714,7516.800000
3,Intel Corporation (NasdaqGS:INTC),59387,62761,70848,71965,77867,79024,63054,23886.400000,26358.014286,28829.628571,31301.242857,33772.857143
4,Kyocera Corporation (TSE:6971),9395.8,10414.5,11040.1,10770.1,10076.3,11949.3,13476.4,15934.900000,16560.810714,17186.721429,17812.632143,18438.542857
...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,Fortive Corporation (NYSE:FTV),5378.2,5756.1,3800.4,4563.9,4634.4,5254.7,5825.7,2715.014286,2723.189286,2731.364286,2739.539286,2747.714286
208,Xiaomi Corporation (SEHK:1810),9357.5,15673.5,23917.4,28145.8,33619,44892.1,38292.4,15973.628571,16425.421429,16877.214286,17329.007143,17780.800000
209,Adient plc (NYSE:ADNT),16583,16391,17393,16304,12582,13312,14340,6073.714286,6454.464286,6835.214286,7215.964286,7596.714286
210,"Resideo Technologies, Inc. (NYSE:REZI)",4455,4519,4827,4988,5071,5846,6370,1614.042857,1730.771429,1847.500000,1964.228571,2080.957143


In [17]:
# create revenue dataframe with all OLS predictions for 2023-2027
df_revenue_complete = []

for x in merged_rev["Company name"]:
    for y in list(merged_rev.columns)[1:]:
        row = [x, y, merged_rev[["Company name", y]][(merged_rev["Company name"] == x)].values[0][1]]
        df_revenue_complete.append(row)

df_revenue_complete = pd.DataFrame(df_revenue_complete, columns = ["Company", "Year", "Revenue"])
df_revenue_complete     

,Company,Year,Revenue
0,ABB Ltd (SWX:ABBN),2016,24929.000000
1,ABB Ltd (SWX:ABBN),2017,25196.000000
2,ABB Ltd (SWX:ABBN),2018,27662.000000
3,ABB Ltd (SWX:ABBN),2019,27978.000000
4,ABB Ltd (SWX:ABBN),2020,26134.000000
...,...,...,...
2539,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2023,30287.385714
2540,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2024,29355.139286
2541,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2025,28422.892857
2542,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2026,27490.646429


In [18]:
# create the actual dataframe with missing values
df_revenue_real = []
for x in df_revenue["Company name"]:
    for y in list(df_revenue.columns)[1:]:
        row = [x, y, df_revenue[["Company name", y]][(df_revenue["Company name"] == x)].values[0][1]]
        df_revenue_real.append(row)

df_revenue_real = pd.DataFrame(df_revenue_real, columns = ["Company", "Year", "Revenue"])
df_revenue_real    

,Company,Year,Revenue
0,ABB Ltd (SWX:ABBN),2016,24929
1,ABB Ltd (SWX:ABBN),2017,25196
2,ABB Ltd (SWX:ABBN),2018,27662
3,ABB Ltd (SWX:ABBN),2019,27978
4,ABB Ltd (SWX:ABBN),2020,26134
...,...,...,...
2539,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2023,8270.25
2540,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2024,9718.26
2541,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2025,11490.58
2542,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2026,10273.06


In [19]:
# substitute missing values in the initial dataframe with ols predictions
for i in range(len(df_revenue_real["Revenue"])):
    if df_revenue_real["Revenue"][i]=="-":
        df_revenue_real["Revenue"][i]=df_revenue_complete["Revenue"][i]

df_revenue_real.to_csv('../clean_data/revenue_complete.csv', index=False)
(df_revenue_real == "-").any(axis=1).sum()

0

## EBITDA - use OLS to predict missing values for 2023-2027

In [20]:
col_names = ['Company name', '2016', '2017', '2018', '2019', '2020', '2021', '2022', "2023", "2024", "2025", "2026", "2027"]

df_ebitda.columns = col_names
df_ebitda
(df_ebitda.iloc[:, :8] == "-").any(axis=1).sum()

0

In [21]:
ols_ebitda_data = df_ebitda[['Company name', '2016', '2017', '2018', '2019', '2020', '2021', '2022']]
(ols_ebitda_data == "-").any(axis=1).sum()

0

In [22]:
new_df_ebitda = []
for x in ols_ebitda_data["Company name"]:
    for y in list(ols_ebitda_data.columns)[1:]:
        row = [x, y, ols_ebitda_data[["Company name", y]][(ols_ebitda_data["Company name"] == x)].values[0][1]]
        new_df_ebitda.append(row)

new_df_ebitda = pd.DataFrame(new_df_ebitda, columns = ["Company", "Year", "Ebitda"])
new_df_ebitda   
(new_df_ebitda == "-").any(axis=1).sum()

0

In [23]:
new_df_ebitda['Year'] = new_df_ebitda['Year'].astype(int)  # Convert 'Year' to integers
new_df_ebitda['Ebitda'] = pd.to_numeric(new_df_ebitda['Ebitda'], errors='coerce')  # Convert 'Value' to numeric, handle non-numeric values as NaN


In [24]:
grouped_ebi = new_df_ebitda.groupby('Company')
ols_models = {}
for name, group in grouped_ebi:
    X = sm.add_constant(group['Year'])  # Add a constant term for the intercept
    y = group['Ebitda']

    # Fit the OLS model
    model = sm.OLS(y, X).fit()

    # Store the model in the dictionary with the company name as the key
    ols_models[name] = model

In [25]:
companies = df_ebitda["Company name"]
companies = companies.tolist()
new_ebi = []
for company, group in grouped_ebi:
    model = ols_models.get(company)
    year_prediction = []
    for year in [2023, 2024, 2025, 2026, 2027]:
        prediction = model.predict([1, year])  # Use .predict() to make predictions
        year_prediction.append(prediction[0])  # The prediction is a one-element array, so extract the value
    new_ebi.append(year_prediction)

new_ebi = pd.DataFrame(new_ebi, columns = ["2023", "2024", "2025", "2026", "2027"])
new_ebi.insert(0, "Company name", companies)
new_ebi
(new_ebi == "-").any(axis=1).sum()

0

In [26]:
merged_ebi = pd.merge(ols_ebitda_data, new_ebi, on='Company name', how='inner')
(merged_ebi == "-").any(axis=1)
merged_ebi

,Company name,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
0,ABB Ltd (SWX:ABBN),2987,2929,3227,3347,2668,4641,4477,-849.700000,-1604.214286,-2358.728571,-3113.242857,-3867.757143
1,"Amazon.com, Inc. (NasdaqGS:AMZN)",12302,15584,27762,36330,48079,59312,54169,4515.857143,4777.821429,5039.785714,5301.750000,5563.714286
2,"Cisco Systems, Inc. (NasdaqGS:CSCO)",14721,14460,15307,16205,15236,15990,15811,1854.000000,1979.910714,2105.821429,2231.732143,2357.642857
3,Intel Corporation (NasdaqGS:INTC),22795,26563,32329,33254,36115,33874,15371,9801.257143,10707.035714,11612.814286,12518.592857,13424.371429
4,Kyocera Corporation (TSE:6971),1230.6,1534,805.7,1423.8,1049.6,1742.2,1789.4,988.585714,1054.003571,1119.421429,1184.839286,1250.257143
...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,Fortive Corporation (NYSE:FTV),1238.1,1334.6,863,936.1,947.8,1224.8,1492.1,343.985714,344.785714,345.585714,346.385714,347.185714
208,Xiaomi Corporation (SEHK:1810),23917.4,816.8,1012.7,1242.8,1510.9,3012.2,1166.9,1661.785714,1863.900000,2066.014286,2268.128571,2470.242857
209,Adient plc (NYSE:ADNT),17393,1068,710,508,421,446,624,1269.142857,1367.642857,1466.142857,1564.642857,1663.142857
210,"Resideo Technologies, Inc. (NYSE:REZI)",393,512,563,359,435,657,790,-44.228571,-72.817857,-101.407143,-129.996429,-158.585714


In [27]:
# create EBITDA dataframe with all OLS predictions for 2023-2027
df_ebitda_complete = []

for x in merged_ebi["Company name"]:
    for y in list(merged_ebi.columns)[1:]:
        row = [x, y, merged_ebi[["Company name", y]][(merged_ebi["Company name"] == x)].values[0][1]]
        df_ebitda_complete.append(row)

df_ebitda_complete = pd.DataFrame(df_ebitda_complete, columns = ["Company", "Year", "Ebitda"])


In [28]:
# create the actual dataframe with missing values
df_ebitda_real = []
for x in df_ebitda["Company name"]:
    for y in list(df_ebitda.columns)[1:]:
        row = [x, y, df_ebitda[["Company name", y]][(df_ebitda["Company name"] == x)].values[0][1]]
        df_ebitda_real.append(row)

df_ebitda_real = pd.DataFrame(df_ebitda_real, columns = ["Company", "Year", "Ebitda"])
df_ebitda_real[df_ebitda_real["Ebitda"]=="-"].count()

Company    214
Year       214
Ebitda     214
dtype: int64

In [29]:
# substitute missing values in the initial dataframe with ols predictions
for i in range(len(df_ebitda_real["Ebitda"])):
    if df_ebitda_real["Ebitda"][i]=="-":
        df_ebitda_real["Ebitda"][i]=df_ebitda_complete["Ebitda"][i]

df_ebitda_real.to_csv('../clean_data/ebitda_complete.csv', index=False)
df_ebitda_real[df_ebitda_real["Ebitda"]=="-"]

,Company,Year,Ebitda


## COGS - use OLS to predict values for 2023-2027

In [30]:
new_column_names = ['Company name', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

df_cogs.columns = new_column_names
df_cogs

,Company name,2016,2017,2018,2019,2020,2021,2022
0,ABB Ltd (SWX:ABBN),17270,17278,19059,19018,18123,19407,19712
1,"Amazon.com, Inc. (NasdaqGS:AMZN)",88265,111934,139156,165536,233307,272344,288831
2,"Cisco Systems, Inc. (NasdaqGS:CSCO)",17663,18121,19225,18575,17206,18918,20242
3,Intel Corporation (NasdaqGS:INTC),22767,23663,27111,29825,34255,35209,36188
4,Kyocera Corporation (TSE:6971),6981.16,7558.96,8281.95,7741.86,7430.67,8595.52,9663.65
...,...,...,...,...,...,...,...,...
260,Fortive Corporation (NYSE:FTV),2685.9,2832.7,1612.6,2066.9,2017.7,2244.8,2462.3
261,Xiaomi Corporation (SEHK:1810),8366.24,13601.32,20882.98,24241.4,28593.63,36925.63,31786.86
262,Adient plc (NYSE:ADNT),14992,15077,16464,15420,11948,12692,13507
266,"Resideo Technologies, Inc. (NYSE:REZI)",3090,3203,3404,3698,3727,4262,4604


In [31]:
new_df_cogs = []
for x in df_cogs["Company name"]:
    for y in list(df_cogs.columns)[1:]:
        row = [x, y, df_cogs[["Company name", y]][(df_cogs["Company name"] == x)].values[0][1]]
        new_df_cogs.append(row)

new_df_cogs = pd.DataFrame(new_df_cogs, columns = ["Company", "Year", "Value"])
new_df_cogs       

,Company,Year,Value
0,ABB Ltd (SWX:ABBN),2016,17270.00
1,ABB Ltd (SWX:ABBN),2017,17278.00
2,ABB Ltd (SWX:ABBN),2018,19059.00
3,ABB Ltd (SWX:ABBN),2019,19018.00
4,ABB Ltd (SWX:ABBN),2020,18123.00
...,...,...,...
1479,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2018,2350.48
1480,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2019,2532.98
1481,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2020,3971.79
1482,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2021,5345.76


In [32]:
new_df_cogs['Year'] = new_df_cogs['Year'].astype(int)  # Convert 'Year' to integers
new_df_cogs['Value'] = pd.to_numeric(new_df_cogs['Value'], errors='coerce')  # Convert 'Value' to numeric, handle non-numeric values as NaN

In [33]:
grouped = new_df_cogs.groupby('Company')

In [34]:
# Iterate over each group and create a regression model
ols_models = {}
for name, group in grouped:
    X = sm.add_constant(group['Year'])  # Add a constant term for the intercept
    y = group['Value']

    # Fit the OLS model
    model = sm.OLS(y, X).fit()

    # Store the model in the dictionary with the company name as the key
    ols_models[name] = model

In [35]:
companies = df_cogs["Company name"]
companies = companies.tolist()

In [36]:
new_cogs = []
for company, group in grouped:
    model = ols_models.get(company)
    year_prediction = []
    for year in [2023, 2024, 2025, 2026, 2027]:
        prediction = model.predict([1, year])  # Use .predict() to make predictions
        year_prediction.append(prediction[0])  # The prediction is a one-element array, so extract the value
    new_cogs.append(year_prediction)

new_cogs = pd.DataFrame(new_cogs, columns = ["2023", "2024", "2025", "2026", "2027"])
new_cogs.insert(0, "Company name", companies)
new_cogs

,Company name,2023,2024,2025,2026,2027
0,ABB Ltd (SWX:ABBN),9879.678571,10122.197857,10364.717143,10607.236429,10849.755714
1,"Amazon.com, Inc. (NasdaqGS:AMZN)",20073.571429,20453.857143,20834.142857,21214.428571,21594.714286
2,"Cisco Systems, Inc. (NasdaqGS:CSCO)",4047.158571,4251.738929,4456.319286,4660.899643,4865.480000
3,Intel Corporation (NasdaqGS:INTC),11640.754286,12747.839643,13854.925000,14962.010357,16069.095714
4,Kyocera Corporation (TSE:6971),13110.488571,13564.551071,14018.613571,14472.676071,14926.738571
...,...,...,...,...,...,...
207,Fortive Corporation (NYSE:FTV),1512.598571,1512.112143,1511.625714,1511.139286,1510.652857
208,Xiaomi Corporation (SEHK:1810),10111.832857,10271.871429,10431.910000,10591.948571,10751.987143
209,Adient plc (NYSE:ADNT),3289.142857,3498.321429,3707.500000,3916.678571,4125.857143
210,"Resideo Technologies, Inc. (NYSE:REZI)",1057.622857,1159.525357,1261.427857,1363.330357,1465.232857


In [37]:
merged_cogs = pd.merge(df_cogs, new_cogs, on='Company name', how='inner')
merged_cogs

,Company name,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
0,ABB Ltd (SWX:ABBN),17270,17278,19059,19018,18123,19407,19712,9879.678571,10122.197857,10364.717143,10607.236429,10849.755714
1,"Amazon.com, Inc. (NasdaqGS:AMZN)",88265,111934,139156,165536,233307,272344,288831,20073.571429,20453.857143,20834.142857,21214.428571,21594.714286
2,"Cisco Systems, Inc. (NasdaqGS:CSCO)",17663,18121,19225,18575,17206,18918,20242,4047.158571,4251.738929,4456.319286,4660.899643,4865.480000
3,Intel Corporation (NasdaqGS:INTC),22767,23663,27111,29825,34255,35209,36188,11640.754286,12747.839643,13854.925000,14962.010357,16069.095714
4,Kyocera Corporation (TSE:6971),6981.16,7558.96,8281.95,7741.86,7430.67,8595.52,9663.65,13110.488571,13564.551071,14018.613571,14472.676071,14926.738571
...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,Fortive Corporation (NYSE:FTV),2685.9,2832.7,1612.6,2066.9,2017.7,2244.8,2462.3,1512.598571,1512.112143,1511.625714,1511.139286,1510.652857
208,Xiaomi Corporation (SEHK:1810),8366.24,13601.32,20882.98,24241.4,28593.63,36925.63,31786.86,10111.832857,10271.871429,10431.910000,10591.948571,10751.987143
209,Adient plc (NYSE:ADNT),14992,15077,16464,15420,11948,12692,13507,3289.142857,3498.321429,3707.500000,3916.678571,4125.857143
210,"Resideo Technologies, Inc. (NYSE:REZI)",3090,3203,3404,3698,3727,4262,4604,1057.622857,1159.525357,1261.427857,1363.330357,1465.232857


### 

In [38]:
# create the final cogs dataframe, by company and  year with OLS forecasts from 2023-2027
cogs_final = []

for x in merged_cogs["Company name"]:
    for y in list(merged_cogs.columns)[1:]:
        row = [x, y, merged_cogs[["Company name", y]][(merged_cogs["Company name"] == x)].values[0][1]]
        cogs_final.append(row)

cogs_final = pd.DataFrame(cogs_final, columns = ["Company", "Year", "Value"])
cogs_final     

,Company,Year,Value
0,ABB Ltd (SWX:ABBN),2016,17270.000000
1,ABB Ltd (SWX:ABBN),2017,17278.000000
2,ABB Ltd (SWX:ABBN),2018,19059.000000
3,ABB Ltd (SWX:ABBN),2019,19018.000000
4,ABB Ltd (SWX:ABBN),2020,18123.000000
...,...,...,...
2539,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2023,35231.068571
2540,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2024,35713.712857
2541,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2025,36196.357143
2542,"Shenzhen Transsion Holdings Co., Ltd. (SHSE:68...",2026,36679.001429


In [39]:
cogs_final.to_csv('../clean_data/cogs_complete.csv', index=False)

In [40]:
unique_companies_new = cogs_final['Company'].unique()
unique_companies_new

array(['ABB Ltd (SWX:ABBN)', 'Amazon.com, Inc. (NasdaqGS:AMZN)',
       'Cisco Systems, Inc. (NasdaqGS:CSCO)',
       'Intel Corporation (NasdaqGS:INTC)',
       'Kyocera Corporation (TSE:6971)',
       'Microsoft Corporation (NasdaqGS:MSFT)',
       'Oracle Corporation (NYSE:ORCL)',
       'Roper Technologies, Inc. (NasdaqGS:ROP)',
       'Sony Group Corporation (TSE:6758)', 'Thales S.A. (ENXTPA:HO)',
       'Apple Inc. (NasdaqGS:AAPL)', 'ASUSTeK Computer Inc. (TWSE:2357)',
       'BorgWarner Inc. (NYSE:BWA)', 'Ciena Corporation (NYSE:CIEN)',
       'CommScope Holding Company, Inc. (NasdaqGS:COMM)',
       'Alphabet Inc. (NasdaqGS:GOOGL)',
       'Intuitive Surgical, Inc. (NasdaqGS:ISRG)',
       'Itron, Inc. (NasdaqGS:ITRI)',
       'Juniper Networks, Inc. (NYSE:JNPR)',
       'Lam Research Corporation (NasdaqGS:LRCX)',
       'Lear Corporation (NYSE:LEA)', 'Mattel, Inc. (NasdaqGS:MAT)',
       'Medtronic plc (NYSE:MDT)',
       'Mettler-Toledo International Inc. (NYSE:MTD)',
       

In [41]:
df = pd.DataFrame(unique_companies_new, columns=['Company'])
df.to_csv('../clean_data/unique_companies.csv', index=False)